Построить классификатор изображений: определение модели автомобиля по фото.  

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import csv
import sys
import os
import math

from ImageDataAugmentor.image_data_augmentor import *
import albumentations

import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, Callback, LearningRateScheduler
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications import EfficientNetB7
import efficientnet.keras as efn
from tensorflow.keras.layers import *

from sklearn.model_selection import train_test_split, StratifiedKFold

import PIL
from PIL import ImageOps, ImageFilter

# #увеличим дефолтный размер графиков
# from pylab import rcParams
# rcParams['figure.figsize'] = 10, 5
# #графики в svg выглядят более четкими
# %config InlineBackend.figure_format = 'svg' 
# %matplotlib inline

In [ ]:
# Обновление tensorflow
! pip install tensorflow==2.4.1
# Загружаем обвязку под keras для использования продвинутых библиотек аугментации
! pip install git+https://github.com/mjkvaak/ImageDataAugmentor
#! pip freeze
#! pip install efficientnet

In [ ]:
# В setup выносим основные настройки

EPOCHS               = 5  # эпох на обучение
BATCH_SIZE           = 48
LR                   = 1e-4
VAL_SPLIT            = 0.15 # выделяем на тест

CLASS_NUM            = 10  # количество классов в нашей задаче
IMG_SIZE             = 224 # какого размера подаем изображения в сеть
IMG_CHANNELS         = 3   # у RGB 3 канала
input_shape          = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

RANDOM_SEED=42
DATA_PATH = '../input/sfcarclassif/'
PATH = "../working/car/" # рабочая директория

## EDA / Анализ данных

In [ ]:
train_df = pd.read_csv(DATA_PATH+'train.csv')
sample_submission = pd.read_csv(DATA_PATH+"sample-submission.csv")
train_df.head()

In [ ]:
train_df.info()

In [ ]:
# распределение классов
train_df.Category.value_counts()

In [ ]:
image = PIL.Image.open(DATA_PATH+'train/train/0/100380.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

## Подготовка данных
Аугментация данных

In [ ]:
AUGMENTATIONS = albumentations.Compose([
    albumentations.HorizontalFlip(p=0.5),
    albumentations.Rotate(limit=30, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=False, p=0.5),
    albumentations.OneOf([
        albumentations.CenterCrop(height=224, width=200),
        albumentations.CenterCrop(height=200, width=224),
    ],p=0.5),
    albumentations.OneOf([
        albumentations.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
        albumentations.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)
    ],p=0.5),
    albumentations.GaussianBlur(p=0.05),
    albumentations.LongestMaxSize(p=0.5),
    albumentations.HueSaturationValue(p=0.5),
    albumentations.RGBShift(p=0.5),
    albumentations.FancyPCA(alpha=0.1, always_apply=False, p=0.5),
#     albumentations.MaskDropout(p=0.5),
    albumentations.CLAHE(p=0.5),
    albumentations.RandomBrightness(limit=0.2, p=0.5),
    albumentations.Resize(IMG_SIZE, IMG_SIZE)
])

In [ ]:
# from ImageDataAugmentor.image_data_augmentor import *

train_gen = ImageDataAugmentor(rescale=1./255,
                        augment=AUGMENTATIONS, 
                        seed=RANDOM_SEED,
                        validation_split=VAL_SPLIT
                       )

train_datagen = train_gen.flow_from_directory(DATA_PATH+'train/train', 
                                            class_mode='categorical', 
                                            batch_size=BATCH_SIZE, 
                                            target_size=(IMG_SIZE, IMG_SIZE),
                                            shuffle=True,
                                            subset='training'
                                           )
test_datagen = train_gen.flow_from_directory(DATA_PATH+'train/train', 
                                             class_mode='categorical', 
                                             batch_size=BATCH_SIZE, 
                                             target_size=(IMG_SIZE, IMG_SIZE),
                                             shuffle=True,
                                             subset='validation'
                                            )

In [ ]:
# посмотрим на результат добавления измененных изображений
train_datagen.show_data(rows=3, cols=5)

# Модель

In [ ]:
base_model = Xception(weights='imagenet', include_top=False, input_shape = input_shape)

In [ ]:
print(len(base_model.layers))

In [ ]:
# в качестве базы можно также взять архитектуру модели EfficientNetB5 (https://www.tensorflow.org/api_docs/python/tf/keras/applications/efficientnet/EfficientNetB5)
# base_model = efn.EfficientNetB5(weights='imagenet', 
#                                 include_top=False, 
#                                 input_shape = input_shape)
# base_model = EfficientNetB7(include_top=False, weights='imagenet', input_shape = input_shape)
# base_model = VGG16(weights='imagenet', include_top=False, input_shape = input_shape)

In [ ]:
base_model.summary()

In [ ]:
# поменять lr
# LR=0.00001

# Устанавливаем новую "голову" (head)
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.25)(x)
# and a logistic layer -- let's say we have 10 classes
predictions = Dense(CLASS_NUM, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])

In [ ]:
model.summary()

## Обучение модели

Добавим ModelCheckpoint чтоб сохранять прогресс обучения модели и можно было потом подгрузить и дообучить модель.

In [ ]:
## настройка lr
# def step_decay(epoch):
#     initial_lrate = 0.1
#     drop = 0.5
#     epochs_drop = 10.0
#     lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
#     return lrate

# learning schedule callback
# lrate = LearningRateScheduler(step_decay)

In [ ]:
checkpoint = ModelCheckpoint('best_model.hdf5' , monitor = ['val_accuracy'] , verbose = 1  , mode = 'max')
callbacks_list = [
    checkpoint,
#     lrate,
]

In [ ]:
# callbacks_list = [
#     tf.keras.callbacks.EarlyStopping(patience=2),
#     tf.keras.callbacks.ModelCheckpoint('best_model.hdf5' , monitor = ['val_accuracy'] , verbose = 1  , mode = 'max'),
#     tf.keras.callbacks.TensorBoard(log_dir='./logs'),
# #     lrate,
# ]

In [ ]:
model.compile(loss="categorical_crossentropy", 
              optimizer=optimizers.Adam(lr=LR), 
              metrics=["accuracy"])

history = model.fit(
        train_datagen,
        steps_per_epoch = train_datagen.samples//train_datagen.batch_size,
        validation_data = test_datagen, 
        validation_steps = test_datagen.samples//test_datagen.batch_size,
        epochs = EPOCHS,
        callbacks = callbacks_list
)

#model.save('../working/model_step4.hdf5') 
model.load_weights('best_model.hdf5') 

In [ ]:
# learn is an instance of Learner class or one of derived classes like ConvLearner
model.lr_find()
model.sched.plot_lr()

In [ ]:
# сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)
model.save('../working/model_last.hdf5')
model.load_weights('best_model.hdf5')

In [ ]:
scores = model.evaluate_generator(test_datagen, steps=len(test_datagen), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

base_model.trainable = True

In [ ]:
base_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = len(base_model.layers)//2

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

In [ ]:
LR=0.0001
model.compile(loss="categorical_crossentropy", 
              optimizer=optimizers.Adam(lr=LR), 
              metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
history = model.fit(
        train_datagen,
        steps_per_epoch = train_datagen.samples//train_datagen.batch_size,
        validation_data = test_datagen, 
        validation_steps = test_datagen.samples//test_datagen.batch_size,
        epochs = EPOCHS,
        callbacks = callbacks_list
)

In [ ]:
# scores = model.evaluate(test_datagen, verbose=1)
# print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# plot_history(history)

In [ ]:
#model.save('../working/model_step2.hdf5')
model.load_weights('best_model.hdf5') 

# Предсказание на тестовых данных

In [ ]:
test_gen = ImageDataAugmentor(rescale=1./255)
test_sub_generator = test_gen.flow_from_dataframe(dataframe=sample_submission,
                                            directory=DATA_PATH+'test/test_upload/',
                                            x_col="Id",
                                            y_col=None,
                                            shuffle=False,
                                            class_mode=None,
                                            target_size=(IMG_SIZE, IMG_SIZE),
                                            batch_size=BATCH_SIZE)

In [ ]:
test_sub_generator.samples

In [ ]:
test_sub_generator.reset()
predictions = model.predict(test_sub_generator, 
                                      steps=len(test_sub_generator), 
                                      verbose=1) 
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_datagen.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]

In [ ]:
filenames_with_dir=test_sub_generator.filenames
submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/','')
submission.to_csv('submission.csv', index=False)
print('Save submit')

In [ ]:
submission.head()